In [1]:
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
set_session(tf.Session(config=config))

Using TensorFlow backend.


In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 8844765174736502951
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 5997789184
locality {
  bus_id: 1
  links {
  }
}
incarnation: 2534072900244040118
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:1e.0, compute capability: 3.7"
]


In [3]:
from tensorflow.contrib.distributions import auto_correlation

In [4]:
import functools
import numpy as np
import pandas as pd

from scipy.special import expit

import sklearn as sk
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

from tensorflow import keras
import tensorflow as tf

def split_companies_train_dev_test(companies):
    "Return train, dev, test set for companies"
    train, test = train_test_split(companies, test_size=0.1, stratify = companies.sector)
    train, dev = train_test_split(train, test_size=0.1, stratify = train.sector)
    return train, dev, test


def filter_stocks(stocks, tickers):
    return stocks.loc[tickers]


def df_to_ts(df):
    res = df.copy()
    res.index = pd.DatetimeIndex(pd.to_datetime(res.date))
    res.drop('date', axis=1)
    return res


def log_softmax(x):
    return x - np.log(np.sum(np.exp(x)))


def sigmoid(x):
    return expit(x)


def sample_correlation(df, window_size=63):
    idx = np.random.randint(0, df.shape[0]-window_size)
    ts = df[idx:idx+window_size]
    fmap = lambda s: ts['pct_return'].corr(ts[s])
    indices = ts.columns.tolist()[1:]
    correlations = np.array(list(map(fmap, indices)))
    return correlations


def create_correlation_score(df, sample_size=1):
    res = np.array([log_softmax(sample_correlation(df)/0.05)
                    for i in range(sample_size)])
    return np.exp(np.nanmean(res, 0))


def load_data(stock_filename=None, indices_filename=None):

    if stock_filename is None:
        stock_filename = '../../data/processed/wiki_stocks_returns.csv'

    if indices_filename is None:
        indices_filename = '../../data/processed/wiki_indices_returns.csv'

    stocks = pd.read_csv(stock_filename, index_col=False) # long format
    indices = pd.read_csv(indices_filename, index_col=False) # wide format

    # Implementation of hierarchical clustering
    drop_column = lambda df,i=0: df.drop(df.columns[i], axis=1)

    stocks = drop_column(stocks)
    stocks = stocks.drop('name', axis=1)
    stocks = stocks.dropna()

    companies = stocks.groupby('ticker').first().reset_index()
    sectors_counts = companies.sector.value_counts()
    sectors_proportions = sectors_counts/sectors_counts.sum()
    sectors_unique = sectors_counts.index.tolist()

    stocks = stocks.set_index('ticker')

    indices_ts = df_to_ts(indices[['date'] + sectors_unique])
    stocks_ts = df_to_ts(stocks.reset_index())

    stocks_all = pd.merge(stocks_ts, indices_ts, 'left')
    stocks_all = stocks_all.dropna() # loss of 200 000 observations
    stocks_all = stocks_all.drop('sector', axis=1)
    stocks_all = stocks_all.groupby('ticker').apply(df_to_ts)
    stocks_all = stocks_all.drop(['ticker', 'date'], axis=1)
    stocks_all = stocks_all.rename(columns={'close': 'pct_return'})

    label_encoder = preprocessing.LabelEncoder()
    label_encoder.fit(sectors_counts.index.tolist())
    ticker_to_sector = dict(zip(companies.ticker, label_encoder.transform(companies.sector)))

    return stocks_all, companies, label_encoder, ticker_to_sector

def sectors_statistics(companies):
    sectors_counts = companies.sector.value_counts()
    sectors_proportions = sectors_counts/sectors_counts.sum()
    sectors_unique = sectors_counts.index.tolist()
    return sectors_counts, sectors_proportions, sectors_unique


def add_common_layers(y):
    y = keras.layers.BatchNormalization()(y)
    y = keras.layers.LeakyReLU()(y)
    return y


def grouped_convolution(y, nb_channels, _strides, cardinality=4):
    # when `cardinality` == 1 this is just a standard convolution
    return keras.layers.SeparableConv1D(nb_channels, kernel_size=3, strides=_strides, padding='same')(y)


def residual_block(y, nb_channels_in, nb_channels_out, cardinality=4, _strides=1, _project_shortcut=False):
    """
    Our network consists of a stack of residual blocks. These blocks have the same topology,
    and are subject to two simple rules:
    - If producing spatial maps of the same size, the blocks share the same hyper-parameters (width and filter sizes).
    - Each time the spatial map is down-sampled by a factor of 2, the width of the blocks is multiplied by a factor of 2.
    """
    shortcut = y
    kl = keras.layers
    # we modify the residual building block as a bottleneck design to make the network more economical
    y = kl.Conv1D(nb_channels_in, kernel_size=1, strides=1, padding='same')(y)
    y = add_common_layers(y)

    # ResNeXt (identical to ResNet when `cardinality` == 1)
    y = grouped_convolution(y, nb_channels_in, _strides=_strides)
    y = add_common_layers(y)

    y = kl.Conv1D(nb_channels_out, kernel_size=1, strides=1, padding='same')(y)
    # batch normalization is employed after aggregating the transformations and before adding to the shortcut
    y = kl.BatchNormalization()(y)

    shortcut = kl.Conv1D(
        nb_channels_out, kernel_size=1, strides=_strides, padding='same')(shortcut)
    shortcut = kl.BatchNormalization()(shortcut)

    y = kl.add([shortcut, y])

    # relu is performed right after each batch normalization,
    # expect for the output of the block where relu is performed after the adding to the shortcut
    y = kl.LeakyReLU()(y)

    return y



# reparameterization trick
# instead of sampling from Q(z|X), sample eps = N(0,I)
# z = z_mean + sqrt(var)*eps
def sampling(args):
    """Reparameterization trick by sampling fr an isotropic unit Gaussian.
    # Arguments:
        args (tensor): mean and log of variance of Q(z|X)
    # Returns:
        z (tensor): sampled latent vector
    """
    z_mean, z_log_var = args
    batch = tf.shape(z_mean)[0]
    dim = keras.backend.int_shape(z_mean)[1]
    # by default, random_normal has mean=0 and std=1.0
    epsilon = tf.random_normal(shape=(batch, dim))
    return z_mean + tf.exp(0.5 * z_log_var) * epsilon

class CovarianceLayer(keras.layers.Layer):

    def __init__(self, num_classes, **kwargs):
        self.num_classes = num_classes
        super(CovarianceLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        # Create a trainable weight variable for this layer.
        super(CovarianceLayer, self).build(input_shape)  # Be sure to call this at the end

    def call(self, inputs):
        series_input, environment_input = inputs
        series_input_multiple = tf.tile(series_input, [1, 1, self.num_classes])
        covariances = tf.reduce_mean(series_input_multiple * environment_input, axis=1)
        
        return covariances

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)


def random_subset(df, window_size=21):
    idx = np.random.randint(0, df.shape[0]-window_size)
    ts = df[idx:idx+window_size]
    return ts


def make_keras_subset(dataset_type, companies_data, stocks_data, label_encoder, batch_size, window_size=21):
    idx = np.random.choice(companies_data[dataset_type].shape[0], batch_size)
    df = companies_data[dataset_type].iloc[idx]

    model_input_data = [random_subset(stocks_data[dataset_type].loc[t], window_size) for t in df.ticker]
    model_series_input = np.array([df['pct_return'].values for df in model_input_data])
    model_series_input = model_series_input.reshape(-1, window_size, 1)

    model_environment_input = np.array([df.iloc[:, 1:].values for df in model_input_data])

    y_true = label_encoder.transform(df.sector)

    return model_series_input, model_environment_input, y_true


class StocksSequence(keras.utils.Sequence):

    def __init__(self, stocks_data,  companies_data, window_size, label_encoder, batch_size):
        self.stocks_data = stocks_data
        self.batch_size = batch_size
        self.label_encoder = label_encoder
        self.companies_data = companies_data
        self.window_size = window_size
        
        _, sectors_proportion, _ = sectors_statistics(companies_data)
        scores = sectors_proportion.shape/sectors_proportion
        scores = scores/scores.sum()
        scores = scores.reset_index(0)
        scores.columns = ['sector', 'sample_prob']
        probs = companies_data.merge(scores, on='sector').sample_prob
        self.companies_sample_probabilities = probs/probs.sum()
        
    def __len__(self):
        return int(np.ceil(self.stocks_data.shape[0] / float(self.batch_size)))

    def __getitem__(self, idx):
        idx = np.random.choice(
            self.companies_data.shape[0], self.batch_size, 
            p=self.companies_sample_probabilities)
        df = self.companies_data.iloc[idx]
        model_input_data = [random_subset(self.stocks_data.loc[t], self.window_size) for t in df.ticker]
        model_input = np.array([df.values for df in model_input_data])
        # correlation = np.array([np.corrcoef(x.T + 0.001*np.random.randn(*x.T.shape))[0, 1:] for x in model_input])
        # model_input = np.transpose(model_input, [0, 2, 1])
        y_true = self.label_encoder.transform(df.sector)
        return model_input, y_true





In [5]:
def make_covariance_convolution_weight(shape, dtype=None):
    kernel_shape = shape[0]
    num_classes = shape[1]
    filters = shape[2]
    x = np.zeros([kernel_shape, num_classes, filters])
    noise = np.random.normal(0, 0.001, size=[kernel_shape, num_classes, filters])

    for j in range(0, filters):
        j_class = j % num_classes
        x[:, [0, j_class], j] = 1
    return x + noise

In [6]:
def normalize_tensor(x):
    """
    x: a B x T x F tensor
    """
    epsilon = 1e-16
    m, s = tf.nn.moments(x, axes=[1], keep_dims=True)
    z = (x - m)/(tf.sqrt(s)+epsilon)
    return z

def correlation_function(x):
    x_normalized = normalize_tensor(x)
    correlations = tf.keras.backend.batch_dot(
        x_normalized, tf.transpose(x_normalized, [0, 2, 1]), axes=[1, 2])
    correlations = correlations/tf.cast(tf.shape(x_normalized)[1], tf.float32)
    return correlations

def correlation_roll(x):
    x_normalized = normalize_tensor(x)
    cross_product = x_normalized[:, :, 0, tf.newaxis] * x_normalized[:, :, 1:]
    cross_product = cross_product/tf.cast(tf.shape(x_normalized)[1], tf.float32)
    return cross_product

In [15]:
def make_model(num_classes=16, window_size=21, latent_dim=32):
    kl = keras.layers
    K = keras.backend
    model_input = keras.layers.Input(
        shape=(window_size, num_classes+1), dtype='float32', name='series_input')
    x_gaussian = kl.GaussianNoise(0.0001)(model_input)
    
    x = kl.Lambda(correlation_roll)(x_gaussian)
    x = kl.AveragePooling1D(10, 5)(x)
    x = kl.Dense(2*num_classes, 'relu')(x)
    x = kl.GlobalAveragePooling1D()(x)
    
    x_tanh = kl.Lambda(lambda y: tf.tanh(tf.scalar_mul(1000, y)))(x_gaussian)
    y = kl.Lambda(correlation_roll)(x_tanh)
    y = kl.AveragePooling1D(10, 5)(y)
    y = kl.Dense(2*num_classes, 'relu')(y)
    y = kl.GlobalAveragePooling1D()(y)
    
    correlation = kl.Lambda(correlation_function)(x_gaussian)
    correlation = kl.Lambda(lambda x: x[:, 0])(correlation)  
    correlation = kl.Dense(2*num_classes, 'relu')(correlation)
    
    x = kl.Concatenate()([x, y, correlation])
    x = kl.Dropout(0.5)(x)
    x = kl.Dense(128, 'relu', name='Embedding')(x)
    x_pred = kl.Dense(num_classes, 'softmax')(x)

    model = keras.Model(inputs = model_input, outputs=[x_pred], name='Classifier')

    return model

In [8]:
# Make train dev test set.
np.random.seed(42)

### Feature engineering

stock_filename = '../data/processed/wiki_stocks_returns.csv'
indices_filename = '../data/processed/wiki_indices_returns.csv'

stocks_all, companies, label_encoder, ticker_to_sector = load_data(stock_filename, indices_filename)
sectors_counts, sectors_proportions, sectors_unique = sectors_statistics(companies)

max_proportion_baseline = sectors_proportions.max()
biggest_sector = sectors_proportions.argmax()

print("Most representated class:", biggest_sector, ', with proportion of ', round(100*max_proportion_baseline, 2), '%.')
# Accuracy of our models should be better than max_proportion_baseline.

companies_data = {}
data_split = split_companies_train_dev_test(companies)
for i, k in enumerate(['train', 'dev', 'test']):
    companies_data[k] = data_split[i]
stocks_data = {k: filter_stocks(stocks_all, v.ticker) for k, v in companies_data.items()}


Most representated class: Financial Services , with proportion of  13.09 %.


In [16]:
window_size = 63
model = make_model(window_size=window_size)

In [11]:
if False:
    model.load_weights('checkpoint/model_weights_thirtieth.json')

In [12]:
def sparse_top_2_categorical_accuracy(y_true, y_pred):
    return keras.metrics.sparse_top_k_categorical_accuracy(y_true, y_pred, k=2)

In [18]:
batch_size = 128

optimizer = keras.optimizers.Adam(0.001)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# model.compile(optimizer=optimizer, loss='mean_squared_error')

print(model.summary())

callbacks = [
    keras.callbacks.ModelCheckpoint('checkpoint/model_weights_thirtythird.json', monitor='val_acc', verbose=1, save_best_only=True, mode='max'),
    keras.callbacks.TensorBoard(log_dir='./logs/thirtythird'),
    keras.callbacks.ReduceLROnPlateau(min_lr=1e-6)
]

stocks_sequence_training = StocksSequence(stocks_data['train'], companies_data['train'], window_size, label_encoder, batch_size)
stocks_sequence_validation = StocksSequence(stocks_data['dev'], companies_data['dev'], window_size, label_encoder, batch_size)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
series_input (InputLayer)       (None, 63, 17)       0                                            
__________________________________________________________________________________________________
gaussian_noise_1 (GaussianNoise (None, 63, 17)       0           series_input[0][0]               
__________________________________________________________________________________________________
lambda_6 (Lambda)               (None, 63, 17)       0           gaussian_noise_1[0][0]           
__________________________________________________________________________________________________
lambda_5 (Lambda)               (None, 63, 16)       0           gaussian_noise_1[0][0]           
__________________________________________________________________________________________________
lambda_7 (

In [19]:
model.fit_generator(
    stocks_sequence_training, steps_per_epoch=1000, epochs=500, callbacks=callbacks, 
    validation_data = stocks_sequence_validation, validation_steps=100, workers=4,
    max_queue_size=20, verbose=1, use_multiprocessing=True, initial_epoch=0)

Epoch 1/500
 999/1000 [============================>.] - ETA: 0s - loss: 2.2104 - acc: 0.3074
Epoch 00001: val_acc improved from -inf to 0.42688, saving model to checkpoint/model_weights_thirtythird.json
1000/1000 [==============================] - 41s 41ms/step - loss: 2.2103 - acc: 0.3075 - val_loss: 1.9027 - val_acc: 0.4269
Epoch 2/500
 998/1000 [============================>.] - ETA: 0s - loss: 1.9385 - acc: 0.3935
Epoch 00002: val_acc improved from 0.42688 to 0.44938, saving model to checkpoint/model_weights_thirtythird.json
1000/1000 [==============================] - 39s 39ms/step - loss: 1.9384 - acc: 0.3935 - val_loss: 1.7938 - val_acc: 0.4494
Epoch 3/500
 998/1000 [============================>.] - ETA: 0s - loss: 1.8736 - acc: 0.4135
Epoch 00003: val_acc improved from 0.44938 to 0.47094, saving model to checkpoint/model_weights_thirtythird.json
1000/1000 [==============================] - 39s 39ms/step - loss: 1.8736 - acc: 0.4135 - val_loss: 1.7469 - val_acc: 0.4709
Epoch 4

Epoch 59/500
 999/1000 [============================>.] - ETA: 0s - loss: 1.5744 - acc: 0.5163
Epoch 00059: val_acc did not improve from 0.57375
1000/1000 [==============================] - 39s 39ms/step - loss: 1.5744 - acc: 0.5164 - val_loss: 1.4879 - val_acc: 0.5691
Epoch 60/500
 999/1000 [============================>.] - ETA: 0s - loss: 1.5604 - acc: 0.5208
Epoch 00060: val_acc did not improve from 0.57375
1000/1000 [==============================] - 39s 39ms/step - loss: 1.5603 - acc: 0.5208 - val_loss: 1.4903 - val_acc: 0.5694
Epoch 61/500
 998/1000 [============================>.] - ETA: 0s - loss: 1.5732 - acc: 0.5161
Epoch 00061: val_acc did not improve from 0.57375
1000/1000 [==============================] - 39s 39ms/step - loss: 1.5732 - acc: 0.5161 - val_loss: 1.4903 - val_acc: 0.5703
Epoch 62/500
 998/1000 [============================>.] - ETA: 0s - loss: 1.5647 - acc: 0.5200
Epoch 00062: val_acc did not improve from 0.57375
1000/1000 [==============================] - 

 999/1000 [============================>.] - ETA: 0s - loss: 1.5708 - acc: 0.5182
Epoch 00119: val_acc did not improve from 0.57375
1000/1000 [==============================] - 39s 39ms/step - loss: 1.5708 - acc: 0.5181 - val_loss: 1.4932 - val_acc: 0.5685
Epoch 120/500
 998/1000 [============================>.] - ETA: 0s - loss: 1.5654 - acc: 0.5182
Epoch 00120: val_acc did not improve from 0.57375
1000/1000 [==============================] - 39s 39ms/step - loss: 1.5653 - acc: 0.5183 - val_loss: 1.4900 - val_acc: 0.5687
Epoch 121/500
 998/1000 [============================>.] - ETA: 0s - loss: 1.5567 - acc: 0.5221
Epoch 00121: val_acc did not improve from 0.57375
1000/1000 [==============================] - 39s 39ms/step - loss: 1.5570 - acc: 0.5220 - val_loss: 1.4897 - val_acc: 0.5700
Epoch 122/500
 999/1000 [============================>.] - ETA: 0s - loss: 1.5633 - acc: 0.5190
Epoch 00122: val_acc did not improve from 0.57375
1000/1000 [==============================] - 39s 39ms/s

 998/1000 [============================>.] - ETA: 0s - loss: 1.5592 - acc: 0.5219
Epoch 00179: val_acc did not improve from 0.57375
1000/1000 [==============================] - 39s 39ms/step - loss: 1.5592 - acc: 0.5219 - val_loss: 1.4938 - val_acc: 0.5680
Epoch 180/500
 998/1000 [============================>.] - ETA: 0s - loss: 1.5678 - acc: 0.5182
Epoch 00180: val_acc did not improve from 0.57375
1000/1000 [==============================] - 39s 39ms/step - loss: 1.5678 - acc: 0.5183 - val_loss: 1.4905 - val_acc: 0.5700
Epoch 181/500
 999/1000 [============================>.] - ETA: 0s - loss: 1.5706 - acc: 0.5164
Epoch 00181: val_acc did not improve from 0.57375
1000/1000 [==============================] - 39s 39ms/step - loss: 1.5708 - acc: 0.5164 - val_loss: 1.4908 - val_acc: 0.5697
Epoch 182/500
 999/1000 [============================>.] - ETA: 0s - loss: 1.5718 - acc: 0.5184
Epoch 00182: val_acc did not improve from 0.57375
1000/1000 [==============================] - 39s 39ms/s

 998/1000 [============================>.] - ETA: 0s - loss: 1.5652 - acc: 0.5185
Epoch 00239: val_acc did not improve from 0.57375
1000/1000 [==============================] - 39s 39ms/step - loss: 1.5652 - acc: 0.5184 - val_loss: 1.4903 - val_acc: 0.5713
Epoch 240/500
 999/1000 [============================>.] - ETA: 0s - loss: 1.5681 - acc: 0.5189
Epoch 00240: val_acc did not improve from 0.57375
1000/1000 [==============================] - 39s 39ms/step - loss: 1.5682 - acc: 0.5189 - val_loss: 1.4964 - val_acc: 0.5686
Epoch 241/500
 997/1000 [============================>.] - ETA: 0s - loss: 1.5650 - acc: 0.5186
Epoch 00241: val_acc did not improve from 0.57375
1000/1000 [==============================] - 39s 39ms/step - loss: 1.5649 - acc: 0.5186 - val_loss: 1.4933 - val_acc: 0.5695
Epoch 242/500
 999/1000 [============================>.] - ETA: 0s - loss: 1.5598 - acc: 0.5211
Epoch 00242: val_acc did not improve from 0.57375
1000/1000 [==============================] - 39s 39ms/s

 998/1000 [============================>.] - ETA: 0s - loss: 1.5706 - acc: 0.5176
Epoch 00299: val_acc did not improve from 0.57375
1000/1000 [==============================] - 39s 39ms/step - loss: 1.5707 - acc: 0.5175 - val_loss: 1.4930 - val_acc: 0.5679
Epoch 300/500
 999/1000 [============================>.] - ETA: 0s - loss: 1.5659 - acc: 0.5183
Epoch 00300: val_acc did not improve from 0.57375
1000/1000 [==============================] - 39s 39ms/step - loss: 1.5659 - acc: 0.5183 - val_loss: 1.4901 - val_acc: 0.5697
Epoch 301/500
 999/1000 [============================>.] - ETA: 0s - loss: 1.5624 - acc: 0.5194
Epoch 00301: val_acc did not improve from 0.57375
1000/1000 [==============================] - 39s 39ms/step - loss: 1.5623 - acc: 0.5194 - val_loss: 1.4932 - val_acc: 0.5676
Epoch 302/500
 998/1000 [============================>.] - ETA: 0s - loss: 1.5601 - acc: 0.5198
Epoch 00302: val_acc did not improve from 0.57375
1000/1000 [==============================] - 39s 39ms/s

 998/1000 [============================>.] - ETA: 0s - loss: 1.5616 - acc: 0.5198
Epoch 00359: val_acc did not improve from 0.57375
1000/1000 [==============================] - 39s 39ms/step - loss: 1.5614 - acc: 0.5198 - val_loss: 1.4900 - val_acc: 0.5719
Epoch 360/500
 999/1000 [============================>.] - ETA: 0s - loss: 1.5655 - acc: 0.5200
Epoch 00360: val_acc did not improve from 0.57375
1000/1000 [==============================] - 39s 39ms/step - loss: 1.5654 - acc: 0.5201 - val_loss: 1.4905 - val_acc: 0.5709
Epoch 361/500
 999/1000 [============================>.] - ETA: 0s - loss: 1.5683 - acc: 0.5163
Epoch 00361: val_acc did not improve from 0.57375
1000/1000 [==============================] - 39s 39ms/step - loss: 1.5683 - acc: 0.5162 - val_loss: 1.4906 - val_acc: 0.5700
Epoch 362/500
 998/1000 [============================>.] - ETA: 0s - loss: 1.5688 - acc: 0.5199
Epoch 00362: val_acc did not improve from 0.57375
1000/1000 [==============================] - 39s 39ms/s

 999/1000 [============================>.] - ETA: 0s - loss: 1.5595 - acc: 0.5205
Epoch 00419: val_acc did not improve from 0.57375
1000/1000 [==============================] - 39s 39ms/step - loss: 1.5594 - acc: 0.5205 - val_loss: 1.4909 - val_acc: 0.5709
Epoch 420/500
 999/1000 [============================>.] - ETA: 0s - loss: 1.5723 - acc: 0.5176
Epoch 00420: val_acc did not improve from 0.57375
1000/1000 [==============================] - 39s 39ms/step - loss: 1.5724 - acc: 0.5176 - val_loss: 1.4901 - val_acc: 0.5697
Epoch 421/500
 998/1000 [============================>.] - ETA: 0s - loss: 1.5632 - acc: 0.5198
Epoch 00421: val_acc did not improve from 0.57375
1000/1000 [==============================] - 39s 39ms/step - loss: 1.5634 - acc: 0.5198 - val_loss: 1.4905 - val_acc: 0.5706
Epoch 422/500
 999/1000 [============================>.] - ETA: 0s - loss: 1.5706 - acc: 0.5175
Epoch 00422: val_acc did not improve from 0.57375
1000/1000 [==============================] - 39s 39ms/s

 998/1000 [============================>.] - ETA: 0s - loss: 1.5721 - acc: 0.5158
Epoch 00479: val_acc did not improve from 0.57375
1000/1000 [==============================] - 39s 39ms/step - loss: 1.5719 - acc: 0.5159 - val_loss: 1.4896 - val_acc: 0.5697
Epoch 480/500
 999/1000 [============================>.] - ETA: 0s - loss: 1.5663 - acc: 0.5189
Epoch 00480: val_acc did not improve from 0.57375
1000/1000 [==============================] - 39s 39ms/step - loss: 1.5662 - acc: 0.5189 - val_loss: 1.4934 - val_acc: 0.5680
Epoch 481/500
 999/1000 [============================>.] - ETA: 0s - loss: 1.5595 - acc: 0.5203
Epoch 00481: val_acc did not improve from 0.57375
1000/1000 [==============================] - 39s 39ms/step - loss: 1.5598 - acc: 0.5203 - val_loss: 1.4899 - val_acc: 0.5697
Epoch 482/500
 998/1000 [============================>.] - ETA: 0s - loss: 1.5604 - acc: 0.5203
Epoch 00482: val_acc did not improve from 0.57375
1000/1000 [==============================] - 39s 39ms/s

In [20]:
stocks_sequence_test = StocksSequence(stocks_data['test'], companies_data['test'], window_size, label_encoder, batch_size)

model.evaluate_generator(stocks_sequence_test, 400, workers=2, use_multiprocessing=True)

[1.4138887017965316, 0.59921875]